# AlphaFold2 Evaluation Metrics

This notebook provides an overview of common metrics used to evaluate AlphaFold2 (AF2) predictions. For each metric, we include a brief description and example code where applicable. You can adapt these code snippets to analyze your own predicted vs. experimental structures.

## 1. Root-Mean-Square Deviation (RMSD)
RMSD measures the average distance between equivalent atoms (typically C<sub>α</sub>) in the predicted and experimental structures.
Below is an example using Biopython to compute RMSD between two PDB files (predicted and experimental).

**Prerequisites:** Install Biopython (`pip install biopython`).

In [2]:
from Bio.PDB import PDBParser, Superimposer

# Paths to PDB files
predicted_pdb = '1ubi.pdb'
experimental_pdb = '1ubi_shifted.pdb'

# Parse structures
parser = PDBParser(QUIET=True)
predicted = parser.get_structure('pred', predicted_pdb)
experimental = parser.get_structure('exp', experimental_pdb)

# Select C-alpha atoms
pred_atoms = [atom for atom in predicted.get_atoms() if atom.get_name() == 'CA']
exp_atoms = [atom for atom in experimental.get_atoms() if atom.get_name() == 'CA']

# Ensure equal length
if len(pred_atoms) != len(exp_atoms):
    raise ValueError('The number of CA atoms in predicted and experimental structures must match.')

# Superimpose and compute RMSD
sup = Superimposer()
sup.set_atoms(exp_atoms, pred_atoms)
print(f"RMSD: {sup.rms:.2f} Å")

RMSD: 0.00 Å


## 2. Global Distance Test (GDT-TS & GDT-HA)
GDT-TS is the percentage of residues whose C<sub>α</sub> atoms fall within 1, 2, 4, and 8 Å after optimal superposition. 
GDT-HA uses more stringent cutoffs at 0.5, 1, 2, and 4 Å.

**Notes:**
- There is no built-in Biopython function for GDT. 
- Typically, software like TM-align or local scripts can compute GDT. Below is a placeholder demonstrating how to call TM-score (which also reports GDT).


In [ ]:
import subprocess

# Paths to PDB files
predicted_pdb = 'predicted_model.pdb'
experimental_pdb = 'experimental_structure.pdb'

# Ensure TM-align or TM-score binary is installed and in PATH
cmd = ['tm_score', experimental_pdb, predicted_pdb]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)

# Look for 'GDT_TS' and 'GDT_HA' in the output.

## 3. Template Modeling Score (TM-score)
TM-score provides a normalized measure of structural similarity (ranging from 0 to 1). Scores > 0.5 generally indicate comparable overall fold.

You can compute TM-score using TM-align or the TM-score binary. Example below reuses the TM-align call.

In [ ]:
import subprocess

# Paths to PDB files
predicted_pdb = 'predicted_model.pdb'
experimental_pdb = 'experimental_structure.pdb'

# Call TM-align (TM-score is part of the output)
cmd = ['TMalign', predicted_pdb, experimental_pdb]
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)

# Parse TM-score from the output lines starting with 'TM-score='.

## 4. Local Distance Difference Test (lDDT)
lDDT evaluates local structural accuracy by comparing distances between atoms in a sliding window.

**Notes:**
- There are Python implementations (e.g., from the `lddt` package) or standalone scripts.
- Below is a placeholder for a custom lDDT function or external tool call.

In [ ]:
def compute_lddt(pred_pdb, exp_pdb, cutoff=15.0, weight_factor=0.0):
    """
    Placeholder function for lDDT calculation. Typically, you'd use an existing implementation or compute pairwise distances within a given cutoff and compare.
    """
    # Implement or call external lDDT calculation here
    pass

# Example usage:
predicted_pdb = 'predicted_model.pdb'
experimental_pdb = 'experimental_structure.pdb'
lddt_score = compute_lddt(predicted_pdb, experimental_pdb)
print(f"lDDT Score: {lddt_score}")

## 5. Predicted Local Distance Difference Test (pLDDT)
AF2 outputs per-residue confidence scores (pLDDT) ranging from 0 to 100, stored in the B-factor column of the PDB.

**Example:** Extracting pLDDT from a PDB file.

In [ ]:
from Bio.PDB import PDBParser

pdb_file = 'af2_predicted_model.pdb'
parser = PDBParser(QUIET=True)
structure = parser.get_structure('af2', pdb_file)

# Collect pLDDT values from B-factor
plddt_values = []
for atom in structure.get_atoms():
    if atom.get_name() == 'CA':
        plddt_values.append(atom.get_bfactor())

import numpy as np
print(f"Average pLDDT: {np.mean(plddt_values):.2f}")
print(f"Per-residue pLDDT values: {plddt_values}")

## 6. Predicted Aligned Error (PAE)
PAE provides a matrix of estimated positional errors between pairs of residues. AF2 outputs a JSON file (`result_model.json`) containing PAE data.

**Example:** Loading PAE from AF2 JSON output.

In [ ]:
import json

json_file = 'result_model.json'
with open(json_file, 'r') as f:
    data = json.load(f)

# PAE is typically under ['predicted_aligned_error'] in the JSON structure
if 'predicted_aligned_error' in data:
    pae_matrix = data['predicted_aligned_error']
    print(f"PAE matrix loaded: size {len(pae_matrix)} x {len(pae_matrix)}")
else:
    print("PAE data not found in the JSON. Check the AF2 output format.")

## 7. MolProbity Score
MolProbity assesses stereochemical quality (rotamer, Ramachandran, clashscore). You can upload a PDB to the MolProbity server or use `phenix.molprobity` tools locally.

**Note:** Below is a placeholder for calling MolProbity locally or instructions to use the web server.

In [ ]:
# Example: placeholder for running MolProbity via command line
# If you have Phenix installed, you can run:
# phenix.molprobity model.pdb

print("Run 'phenix.molprobity model.pdb' to obtain a MolProbity report.")

----
### Conclusion
This notebook provided a starting point for computing and interpreting various AF2 evaluation metrics. You can expand each section with more detailed implementations, visualization of results, and batch-processing scripts for large datasets.